# **MODEL REGISTRY, TRACKING AND MLflow DEPLOYMENT**

In [1]:
# necessary libs
!pip install hyperopt
!pip install mlflow
!pip install pyngrok

  Using cached pyngrok-7.2.8-py3-none-any.whl.metadata (10 kB)
Using cached pyngrok-7.2.8-py3-none-any.whl (25 kB)


In [2]:
# importing them - for ml flow/ml
import os
import argparse
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

#loading libs for the server
import pyngrok
from pyngrok import ngrok, conf
import subprocess
import getpass

# for hyper-parameter tuning
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
# Reading the data
data_url = "https://raw.githubusercontent.com/aniruddhachoudhury/Red-Wine-Quality/refs/heads/master/winequality-red.csv"

df = pd.read_csv(data_url)

In [4]:
#Function for performance evaluation
def evaluate(y,pred):
    rmse = np.sqrt(mean_squared_error(y,pred))
    mae = mean_absolute_error(y,pred)
    r2 = r2_score(y,pred)

    return rmse, mae, r2

In [5]:
# dataset prep
train,test = train_test_split(df,random_state=123)
train_x = train.drop(["quality"],axis=1)
test_x = test.drop(["quality"],axis=1)

train_y = train[["quality"]]
test_y = test[["quality"]]

In [6]:
#MLflow config
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("hyperparameter-experiment")

2025/05/17 22:33:16 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/05/17 22:33:16 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/content/mlruns/1', creation_time=1747521198773, experiment_id='1', last_update_time=1747521198773, lifecycle_stage='active', name='hyperparameter-experiment', tags={}>

In [7]:
# ngrok set up
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel "https://debc-34-138-142-218.ngrok-free.app" -> "http://127.0.0.1:5000"


In [8]:
#best model
params= {"alpha":1.0404948264732425,
"copy_X":True,
"fit_intercept":True,
"l1_ratio":0.6823008984208807,
"max_iter":1000,
"normalize":"deprecated",
"positive":False,
"precompute":False,
"random_state":None,
"selection":"cyclic",
"tol":0.0001,
"warm_start":False}

In [9]:
# with auto log
mlflow.sklearn.autolog()
with mlflow.start_run() :
    # Remove the 'normalize' key as it's not a valid argument in recent scikit-learn versions
    if 'normalize' in params:
        del params['normalize']
    lr = ElasticNet(**params)
    lr.fit(train_x,train_y)

    pred = lr.predict(test_x)

    rmse,mae,r2 = evaluate(test_y,pred)

    # The variables alpha and l1_ratio are not defined in this scope.
    # Access them from the params dictionary instead.
    print(f"Elastic net Params: alpha: {params['alpha']}, l1_ratio: {params['l1_ratio']}")
    print(f"Elastic net metric: rmse:{rmse}, mae:{mae},r2:{r2}")

Elastic net Params: alpha: 1.0404948264732425, l1_ratio: 0.6823008984208807
Elastic net metric: rmse:0.7896212045750471, mae:0.6491379913030703,r2:0.019034539467356226


In [13]:
#adding some extra models
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "Elasticnet")
        mlflow.log_params(params)

        lr = ElasticNet(**params)
        lr.fit(train_x,train_y)

        pred = lr.predict(test_x)

        rmse,mae,r2 = evaluate(test_y,pred)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae",mae)
        mlflow.log_metric("r2",r2)

    return {'loss': rmse, 'status': STATUS_OK}

In [14]:
search_space = { "alpha": hp.loguniform('alpha',0.01,1),
                  "l1_ratio": hp.uniform('l1_ratio',0,1)}

In [15]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

100%|██████████| 10/10 [00:51<00:00,  5.15s/trial, best loss: 0.7321955075384519]


In [17]:
# connecting to server

from mlflow.tracking import MlflowClient
#MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [19]:
# interacting with tracking server
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/content/mlruns/1', creation_time=1747521198773, experiment_id='1', last_update_time=1747521198773, lifecycle_stage='active', name='hyperparameter-experiment', tags={}>,
 <Experiment: artifact_location='/content/mlruns/0', creation_time=1747521198723, experiment_id='0', last_update_time=1747521198723, lifecycle_stage='active', name='Default', tags={}>]

In [25]:
from mlflow.entities import ViewType
#selecting all models with rmse < 0.75
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse <0.75",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse DESC"]
)

In [26]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ac0c559e96384d31b5f89ce3a9815152, rmse: 0.7390
run id: 03dc5e296d7a4c50961e40c586643248, rmse: 0.7322


In [27]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [28]:
# registering the model
run_id = "ac0c559e96384d31b5f89ce3a9815152"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="wine model")

Registered model 'wine model' already exists. Creating a new version of this model...
Created version '2' of model 'wine model'.


<ModelVersion: aliases=[], creation_timestamp=1747522285771, current_stage='None', description=None, last_updated_timestamp=1747522285771, name='wine model', run_id='ac0c559e96384d31b5f89ce3a9815152', run_link=None, source='/content/mlruns/1/ac0c559e96384d31b5f89ce3a9815152/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [30]:
model_name = "wine model"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: None


<ipython-input-30-9a09c010feec>:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


# **Testing a model in production**

In [31]:
#df = lets just re-use the same df but a new split
train,test = train_test_split(df,random_state=25)
train_x = train.drop(["quality"],axis=1)
test_x = test.drop(["quality"],axis=1)

train_y = train[["quality"]]
test_y = test[["quality"]]

In [36]:
# Get the latest version of the registered model
latest_versions = client.get_latest_versions(name=model_name)
if latest_versions:
    latest_version = latest_versions[0].version
    print(f"Latest version of '{model_name}': {latest_version}")

    # Transition the latest version to the "Production" stage
    client.transition_model_version_stage(
        name=model_name,
        version=latest_version,
        stage="Production"
    )
    print(f"Transitioned version {latest_version} of '{model_name}' to 'Production' stage.")
else:
    print(f"No versions found for model '{model_name}'.")

Latest version of 'wine model': 2
Transitioned version 2 of 'wine model' to 'Production' stage.


<ipython-input-36-8a10b1307279>:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)
<ipython-input-36-8a10b1307279>:8: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [39]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    # Calculate RMSE by taking the square root of the mean squared error.
    # This avoids the 'squared' keyword argument which might not be supported
    # by the scikit-learn version in the environment.
    return {"rmse": np.sqrt(mean_squared_error(y_test, y_pred))}

In [40]:
test_model(name=model_name, stage="production", X_test=test_x, y_test=test_y)

{'rmse': np.float64(0.719291983377612)}

# **DEPLOYMENT IN GCP**

In [ ]:
## Following setps involved to setup MLFlow on GCP:
# -Create a PostgreSQL DB for storing model metadata.
# -Create a Google Cloud Storage Bucket for storing artifacts.
# -Create a Compute Engine instance to install MLFlow and run the MLFlow server
# -SSH into Compute machine using the UI and run following commands: (Navigate to the compute engine and ssh into it. tcp port (under firewall) might need changing);
#  sudo apt update
#  pip3 install mlflow psycopg2-binary
#  mlflow server -h 0.0.0.0 -p 5000 --backend-store-uri postgresql://DB_USER:DB_PASSWORD@DB_ENDPOINT:5432/DB_NAME --default-artifact-root gs://GS_BUCKET_NAME '''
#  copy the external ip and add :5000/ this should open the mlflow UI

In [ ]:
#mlflow server --backend-store-uri sqlite:///mlflow.db
mlflow.set_tracking_uri("http://34.135.17.37:5000/")
mlflow.set_experiment("Wine-Quality-on-gcp")

#might need google cloud authentication